<a href="https://colab.research.google.com/github/sviii09/ChronoLens-policy-intelligence/blob/main/rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kagglehub langchain langchain-community langchain-google-genai faiss-cpu sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.4/719.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.47.0 which is incompatible.
google-colab

In [ ]:
import kagglehub
import os
import pandas as pd

path = kagglehub.dataset_download("jainamgada45/indian-government-schemes")
print("Dataset path:", path)

# list files
os.listdir(path)


100%|██████████| 3.10M/3.10M [00:00<00:00, 128MB/s]

Extracting files...


Dataset path: /root/.cache/kagglehub/datasets/jainamgada45/indian-government-schemes/versions/1


['updated_data.csv']

In [ ]:
df = pd.read_csv(os.path.join(path, os.listdir(path)[0]))
df.head()


,scheme_name,slug,details,benefits,eligibility,application,documents,level,schemeCategory,Unnamed: 9,tags
0,"""Immediate Relief Assistance"" under ""Welfare a...",ira-wrflsncs,"The scheme ""Immediate Relief Assistance"" is a ...","₹ 1,00,000, in two installments of ₹ 50,000 ea...",The applicant should be the family (legal heir...,Step 1: The interested applicant should visit ...,Photograph of the Family (Legal Heir) of the M...,State,"Agriculture,Rural & Environment, Social welfar...",NaN,"Missing, Fisherman, Relief, Financial Assistan..."
1,AICTE SHORT TERM TRAINING PROGRAMME-SFURTI SCHEME,astpss,"Short Term Training Programme-SFURTI Program, ...","Financial Assistance : Limit of funding ₹ 4,00...",The institution should be AICTE approved.,Registration of New Institute: Step 01: Visit ...,Feedback Form Copy of Proceedings Completion R...,Central,Education & Learning,NaN,"Trainings, Financial Assistance, AICTE"
2,Burial and Ex-gratia Payment Scheme in Case of...,baepsicodouldwact,"Launched in 2014, the "" Burial and Ex-gratia P...","Funeral Assistance: ₹3,000 payable in case of ...",The deceased construction worker should have b...,Step 1: The interested applicant should visit ...,Aadhaar Card of the applicant (nominee/Legal h...,State,Social welfare & Empowerment,NaN,"Building Worker, Construction Workers, Unregis..."
3,Consortia & Tender Marketing Scheme,ctms,Promotion of the product of Micro and Small En...,Enlistment of the Unit for participating in Go...,Micro & Small Enterprises registered with NSIC...,"Step 01: The application form, in the prescrib...",A passport size photograph of each of the Prop...,Central,Business & Entrepreneurship,NaN,"Goods And Services Marketing, Turnkey Projects..."
4,Garuda Scheme for Funeral Expense,gsfe,Andhra Pradesh Brahmin Welfare Corporation (AB...,"Financial Assistance of ₹10,000/- for funeral ...",The applicant should be a close relative of th...,Registration and apply Step 01: Applicants hav...,Passport-size Photograph of the Applicant Aadh...,State,Social welfare & Empowerment,NaN,"Social Welfare, Financial Assistance, Deceased..."


In [ ]:
df = df.drop(columns=[col for col in df.columns if "Unnamed" in col], errors="ignore")
df.columns


Index(['scheme_name', 'slug', 'details', 'benefits', 'eligibility',
       'application', 'documents', 'level', 'schemeCategory', 'tags'],
      dtype='object')

In [ ]:
from langchain_core.documents import Document


documents = []

for _, row in df.iterrows():
    content = f"""
    Scheme Name: {row['scheme_name']}
    Category: {row['schemeCategory']}
    Level: {row['level']}
    Tags: {row['tags']}

    Details:
    {row['details']}

    Benefits:
    {row['benefits']}

    Eligibility:
    {row['eligibility']}

    Application Process:
    {row['application']}

    Required Documents:
    {row['documents']}
    """

    documents.append(
        Document(
            page_content=content.strip(),
            metadata={
                "scheme_name": row["scheme_name"],
                "category": row["schemeCategory"],
                "level": row["level"],
                "tags": row["tags"],
                "slug": row["slug"]
            }
        )
    )

len(documents)


3400

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=120
)

docs = text_splitter.split_documents(documents)

print("Total chunks:", len(docs))


Total chunks: 40083


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


/tmp/ipython-input-3055314890.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_community.vectorstores import FAISS

texts = [d.page_content for d in docs]
metadatas = [d.metadata for d in docs]

vectorstore = FAISS.from_texts(
    texts=texts,
    embedding=embeddings,
    metadatas=metadatas
)

# SAVE so you don’t redo this again like a clown
vectorstore.save_local("govt_schemes_faiss")

print("FAISS index built & saved")


FAISS index built & saved


In [ ]:
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5}
)


In [ ]:
!pip install -q langchain


In [ ]:
!pip install -q transformers accelerate torch


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    temperature=0.1,
    do_sample=False
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
from langchain_community.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=llm_pipeline)


/tmp/ipython-input-2277194983.py:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("""
You are an assistant answering questions about Indian government schemes.
Use ONLY the provided context.

Context:
{context}

Question:
{question}

Answer clearly and factually.
""")

rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
response = rag_chain.invoke(
    "Which schemes are available for women entrepreneurs and what benefits do they provide?"
)

print(response)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Human: 
You are an assistant answering questions about Indian government schemes.
Use ONLY the provided context.

Context:
[Document(id='32945a44-6299-42d4-8f87-22aa2454563a', metadata={'scheme_name': 'Mukhyamantri Mahila Utkarsh Yojana', 'category': 'Social welfare & Empowerment, Women and Child', 'level': 'State', 'tags': 'Mukhyamantri, Mahila, Utkarsh Yojana', 'slug': 'mmuy'}, page_content='them to become financially independent, contributing to their personal and professional growth. No Income Limit: There is no income limit for women entrepreneurs to apply for the scheme, making it accessible to women from all income groups. \ufeff'), Document(id='3d20e9a5-06e9-4fc0-86a7-c15ee49af849', metadata={'scheme_name': 'Incentives to Women Entrepreneurs Scheme, 2008', 'category': 'Business & Entrepreneurship', 'level': 'State', 'tags': 'Incentive, Woman, Entrepreneur, Subsidy, Industry', 'slug': 'iwes'}, page_content='of this scheme were eligible. Women entrepreneurs were given preference 